In [1]:
from qiskit.dagcircuit import DAGCircuit
from qiskit.circuit import QuantumCircuit, QuantumRegister, Gate,ParameterVector
from qiskit.circuit.library import CXGate, ECRGate
from qiskit.transpiler import PassManager
from qiskit.transpiler.basepasses import TransformationPass
from qiskit.quantum_info import Operator, pauli_basis
from qiskit.quantum_info import SparsePauliOp
from qiskit_ibm_runtime.fake_provider import FakeMumbaiV2
from qiskit_ibm_runtime import EstimatorV2 as Estimator, QiskitRuntimeService
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, QuantumError, coherent_unitary_error
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import Pauli, SparsePauliOp

import sys
sys.path.append("../")
from clapton.hamiltonians import ising_model

import numpy as np
from typing import Iterable, Optional
from scipy.optimize import minimize

In [2]:
class PauliTwirl(TransformationPass):
    """Add Pauli twirls to two-qubit gates."""
 
    def __init__(
        self,
        gates_to_twirl: Optional[Iterable[Gate]] = None,
    ):
        """
        Args:
            gates_to_twirl: Names of gates to twirl. The default behavior is to twirl all
                two-qubit basis gates, `cx` and `ecr` for IBM backends.
        """
        if gates_to_twirl is None:
            gates_to_twirl = [CXGate(), ECRGate()]
        self.gates_to_twirl = gates_to_twirl
        self.build_twirl_set()
        super().__init__()
 
    def build_twirl_set(self):
        """
        Build a set of Paulis to twirl for each gate and store internally as .twirl_set.
        """
        self.twirl_set = {}
 
        # iterate through gates to be twirled
        for twirl_gate in self.gates_to_twirl:
            twirl_list = []
 
            # iterate through Paulis on left of gate to twirl
            for pauli_left in pauli_basis(2):
                # iterate through Paulis on right of gate to twirl
                for pauli_right in pauli_basis(2):
                    # save pairs that produce identical operation as gate to twirl
                    if (Operator(pauli_left) @ Operator(twirl_gate)).equiv(
                        Operator(twirl_gate) @ pauli_right
                    ):
                        twirl_list.append((pauli_left, pauli_right))
 
            self.twirl_set[twirl_gate.name] = twirl_list
 
    def run(
        self,
        dag: DAGCircuit,
    ) -> DAGCircuit:
        # collect all nodes in DAG and proceed if it is to be twirled
        twirling_gate_classes = tuple(
            gate.base_class for gate in self.gates_to_twirl
        )
        for node in dag.op_nodes():
            if not isinstance(node.op, twirling_gate_classes):
                continue
 
            # random integer to select Pauli twirl pair
            pauli_index = np.random.randint(
                0, len(self.twirl_set[node.op.name])
            )
            twirl_pair = self.twirl_set[node.op.name][pauli_index]
 
            # instantiate mini_dag and attach quantum register
            mini_dag = DAGCircuit()
            register = QuantumRegister(2)
            mini_dag.add_qreg(register)
 
            # apply left Pauli, gate to twirl, and right Pauli to empty mini-DAG
            mini_dag.apply_operation_back(
                twirl_pair[0].to_instruction(), [register[0], register[1]]
            )
            mini_dag.apply_operation_back(node.op, [register[0], register[1]])
            mini_dag.apply_operation_back(
                twirl_pair[1].to_instruction(), [register[0], register[1]]
            )
 
            # substitute gate to twirl node with twirling mini-DAG
            dag.substitute_node_with_dag(node, mini_dag)
 
        return dag

In [3]:
def qiskit_circular_ansatz(N, reps=1): 
    qc = QuantumCircuit(N)

    # define your parameters
    p = ParameterVector('p', (N*2) *(reps+1)) 

    for r in range(reps): 
        for i in range(N):
            qc.ry(p[2*N*r+i], i)  
        for i in range(N):
            qc.rz(p[2*N*r+(i+N)], i)
        for i in range(N):
            control = (i-1) % N
            target = i
            qc.cx(control, target)
    for i in range(N):
        qc.ry(p[2*N*reps+i], i)
    for i in range(N):
        qc.rz(p[2*N*reps + (i+N)], i)
    return qc
num_qubits = 5
reps = 1
qc = qiskit_circular_ansatz(num_qubits,reps)

In [4]:
# Multi Qubit Hamiltonian
coeffs,paulis,_ = ising_model(N=num_qubits,Jx=0.2,Jy=0.3,h=0.4)

weights  =  coeffs
pauli_op = [([pauli,weight]) for pauli,weight in zip(paulis,weights)]
hamiltonian = SparsePauliOp.from_list([ op for op in pauli_op ])


In [5]:
# Set initial parameters
initial_params = np.zeros(qc.num_parameters)
initial_params

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.])

In [6]:
# Calculate Exact Ground State Energy
from qiskit_algorithms import NumPyMinimumEigensolver

numpy_solver = NumPyMinimumEigensolver()
ref_result = numpy_solver.compute_minimum_eigenvalue(operator=hamiltonian)
ref_value = ref_result.eigenvalue.real
print(f"Reference value: {ref_value:.5f}") #NOTE: Check if both reference values match

Reference value: -2.08662


In [7]:
# Create your custom pass
pm = PassManager([PauliTwirl()])

#Create your twirled circuits
twirled_qcs = [pm.run(qc) for _ in range(100)]

In [8]:
# Device Noise Model
backend = FakeMumbaiV2() # Your quantum backend
noise_model = NoiseModel.from_backend(backend)
print(noise_model)

NoiseModel:
  Basis gates: ['cx', 'delay', 'id', 'measure', 'reset', 'rz', 'sx', 'x']
  Instructions with noise: ['reset', 'measure', 'cx', 'id', 'x', 'sx']
  Qubits with noise: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
  Specific qubit errors: [('reset', (0,)), ('reset', (1,)), ('reset', (2,)), ('reset', (3,)), ('reset', (4,)), ('reset', (5,)), ('reset', (6,)), ('reset', (7,)), ('reset', (8,)), ('reset', (9,)), ('reset', (10,)), ('reset', (11,)), ('reset', (12,)), ('reset', (13,)), ('reset', (14,)), ('reset', (15,)), ('reset', (16,)), ('reset', (17,)), ('reset', (18,)), ('reset', (19,)), ('reset', (20,)), ('reset', (21,)), ('reset', (22,)), ('reset', (23,)), ('reset', (24,)), ('reset', (25,)), ('reset', (26,)), ('cx', (0, 1)), ('cx', (1, 0)), ('cx', (1, 2)), ('cx', (1, 4)), ('cx', (2, 1)), ('cx', (2, 3)), ('cx', (3, 2)), ('cx', (3, 5)), ('cx', (4, 1)), ('cx', (4, 7)), ('cx', (5, 3)), ('cx', (5, 8)), ('cx', (6, 7)), ('cx', (7, 4)

In [9]:
# Only Coherent Noise Model 
noise_model = NoiseModel()

# Define the over-rotation angle (in radians)
theta = 1e-2 * np.pi/2 # Adjust this value to control the amount of over-rotation

# Create the over-rotated CX gate
cx_overrotated = Operator([
    [np.cos(theta), -1j*np.sin(theta), 0, 0],
    [-1j*np.sin(theta), np.cos(theta), 0, 0],
    [0, 0, np.cos(theta), -1j*np.sin(theta)],
    [0, 0, -1j*np.sin(theta), np.cos(theta)]
])

# Create a quantum error from the over-rotated CX gate
noise_model.add_all_qubit_quantum_error((cx_overrotated), ['cx'])
print(noise_model)

NoiseModel:
  Basis gates: ['cx', 'id', 'rz', 'sx']
  Instructions with noise: ['cx']
  All-qubits errors: ['cx']


In [10]:
# ##Noisy
estimator = Estimator(
    mode=AerSimulator(
        noise_model=noise_model,
        coupling_map=backend.coupling_map,
        basis_gates=noise_model.basis_gates,
        device = 'GPU'
    )
)

# # estimator = Estimator(mode=AerSimulator(method='statevector',device = "GPU"))

In [11]:

# Calculate expectation value
energy_values = np.array([])
for transpiled_circuit in twirled_qcs:
    transpiled_circuit = transpiled_circuit.assign_parameters(initial_params)
    job = estimator.run([(transpiled_circuit, hamiltonian)]) #Something wrong w parameter optim.
    result = job.result()
    energy = result[0].data.evs #NOTE: Is this actually expectation value?
    energy_values = np.append(energy_values, energy)

# Print statement to see where we are at
# print(f"Iteration: {iteration_number}, Current energy: {np.average(energy_values)}, Current parameters: {initial_params}")
# return result.values[0]
np.average(energy_values)

1.9997353515625

In [12]:
non_pt_circ = qc.assign_parameters(initial_params)
job = estimator.run([(non_pt_circ, hamiltonian)])
result = job.result()
energy = result[0].data.evs
energy

array(1.99487305)